# Automated Search + cleaning process

Download the starter guide Jupyter notebook [here](https://learn.aylien.com/download/news_api_python_starter_guide.ipynb)  
Read the documentation [here](https://docs.aylien.com/newsapi/#getting-started)

## 1. Configuring Your API Connection<a class="anchor" id="Configuring_Your_API_Connection"></a>
 If you don't have any credentials yet, you can sign up for a free trial [here](https://newsapi.aylien.com/signup).

In [1]:
from __future__ import print_function

import os
import requests
import datetime
from datetime import date
from dateutil.tz import tzutc
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from pprint import pprint

#for sql handling
import psycopg2
import sql
from sql import engine
from sql import get_data

#put credentials here
headers = {
    'X-AYLIEN-NewsAPI-Application-ID': '', 
    'X-AYLIEN-NewsAPI-Application-Key': ''
}

print('Complete')

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2


.env file found and working
Complete


## 2. Define Functions to Pull Data
The Functions below will be used to pull the data from the API using get requests. In some cases, data will be returned as an array of objects e.g. the get_stories function. In others data will be returned as Pandas dataframes e.g. the get_timeseires function.

In [2]:
#=======================================================================================
def get_timeseries(params, print_params = None, print_count = None):
    if print_params is None or print_params == 'yes':
        pprint(params)
    
    response = requests.get('https://api.aylien.com/news/time_series', params=params, headers=headers).json()
    
    if 'errors' in response or 'error' in response:
        pprint(response)
    
    #convert to dataframe
    timeseries_data = pd.DataFrame(response['time_series'])
    
    # convert back to datetime
    timeseries_data['published_at'] = pd.to_datetime(timeseries_data['published_at'])
    
    timeseries_data['published_at'] = timeseries_data['published_at'].dt.date
    
    if print_count is None  or print_count == 'yes':
        print('Number of stories returned : ' + str(format(timeseries_data['count'].sum(), ",d")))
    
    return timeseries_data


#=======================================================================================
def get_stories(params, print_params = None, print_count = None, print_story = None):
    if print_params is None or print_params == 'yes':
        pprint(params)
    
    fetched_stories = []
    stories = None
    while stories is None or len(stories) > 0:
        try:
            response = requests.get('https://api.aylien.com/news/stories', params=params, headers=headers).json()
        except Exception as e:
            continue
            
        if 'errors' in response or 'error' in response:
            pprint(response)
        
        stories = response['stories']
        
        if len(stories) > 0:
            print(stories[0]['title'])
            print(stories[0]['links']['permalink'])
        
        params['cursor'] = response['next_page_cursor']
        
        fetched_stories += stories
        
        
        if (print_story is None or print_story == 'yes') and len(stories) > 0:
            pprint(stories[0]['title'])
        
        if print_count is None  or print_count == 'yes':
            print("Fetched %d stories. Total story count so far: %d" %(len(stories), len(fetched_stories)))
        
    return fetched_stories

#=======================================================================================
def get_top_ranked_stories(params, no_stories, print_params = None, print_count = None):
    if print_params is None or print_params == 'yes':
        pprint(params)
    
    fetched_stories = []
    stories = None
    while stories is None or len(stories) > 0 and len(fetched_stories) < no_stories:
        try:
            response = requests.get('https://api.aylien.com/news/stories', params=params, headers=headers).json()
        except Exception as e:
            continue
            
        if 'errors' in response or 'error' in response:
            pprint(response)
        
        stories = response['stories']
        
        if len(stories) > 0:
            print(stories[0]['title'])
            print(stories[0]['links']['permalink'])
        
        params['cursor'] = response['next_page_cursor']
        
        fetched_stories += stories
        
        if print_count is None  or print_count == 'yes':
            print("Fetched %d stories. Total story count so far: %d" %(len(stories), len(fetched_stories)))
        
    return fetched_stories


#=======================================================================================
def get_clusters(params={}):
    #pprint(params)
    
    response = requests.get('https://api.aylien.com/news/clusters', params=params, headers=headers).json()
    
    if 'errors' in response or 'error' in response:
        pprint(response)

    return response


#=======================================================================================
# pull trends data to identify most frequently occuring entities or keywords   
def get_trends(params={}):
    #pprint(params)
    
    response = requests.get('https://api.aylien.com/news/trends', params=params, headers=headers).json()
    
    if 'errors' in response or 'error' in response:
        pprint(response)
    
    return response

#=======================================================================================
def get_cluster_from_trends(params, print_params = None):
    
    if print_params is None or print_params == 'yes':
        pprint(params)
    
    """
    Returns a list of up to 100 clusters that meet the parameters set out.
    """
    response = requests.get('https://api.aylien.com/news/trends', params=params, headers=headers).json()
    
    if 'errors' in response or 'error' in response:
        pprint(response)
    
    if len(response) > 0:
        return response["trends"]

    
#=======================================================================================
# identify the top ranked story per cluster
def get_top_stories_in_cluster(cluster_id, no_stories):    
    top_story_params = {
                        'clusters[]' : [cluster_id]
                        , 'sort_by' : "source.rankings.alexa.rank"
                        , 'per_page' : no_stories
                        , 'return[]' : ['id', 'language', 'links', 'title', 'source', 'translations', 'clusters', 'published_at']
                        }
    
    response = requests.get('https://api.aylien.com/news/stories', params=top_story_params, headers=headers).json()
    
    if 'errors' in response or 'error' in response:
        pprint(response)
    if len(response["stories"]) > 0:
        return response["stories"]
    else:
        return None
    
    
#=======================================================================================
# helper endpoint that takes a string of characters and an entity type (such as sources, or DBpedia entities) and returns matching entities of the specified type along with additional metadata
# params = {'type' : 'source_names', 'term' : 'Times of India' } 
    
def autocompletes(params={}):
    pprint(params)
    """
    Returns a list of up to 100 clusters that meet the parameters set out.
    """
    response = requests.get('https://api.aylien.com/news/autocompletes', params=params, headers=headers).json()
    
    if 'errors' in response or 'error' in response:
        pprint(response)
    
    pprint(response)

### 2.1 Define Other Useful Functions
These other functions will help us format data as necessary.

In [3]:
# return transalted title or body of a story (specify in params)   
def return_translated_content(story_x, text_x):
    if 'translations' in story_x:
        return story_x['translations']['en'][text_x]
    else:
        return story_x[text_x]
    
    
# create smaller lists from big lists
def chunks(lst, n):
    return list(lst[i:i + n] for i in range(0, len(lst), n))


#=======================================================================================
# split title string over multiple lines for legibility on graph
def split_title_string(dataframe_x, column_x):
    title_strings = []

    for index, row in dataframe_x.iterrows():
        word_array = row[column_x].split()
        counter = 0
        string = ''
        for word in word_array:
            if counter == 7:
                string += (word + '<br>')
                counter = 0
            else:
                string += (word + ' ')
                counter += 1
        title_strings.append(string)

    dataframe_x[column_x + '_string'] = (title_strings)


#=======================================================================================
def print_keyword_mention(story_x, element_x, keyword_x):
    body_x = story[element_x]
    
    if 'translations' in story and story['translations'] is not None and 'en' in story['translations']:
        body_x = story['translations']['en'][element_x]
    
    # extract a window around key entity
    e_idx = body_x.find(keyword_x)
    e_end = e_idx + len(keyword_x)
    if e_idx >= 0:
        e_str = body_x[e_idx-100:e_idx] + "\033[1m" + body_x[e_idx:e_end] + "\033[0m " + body_x[e_end+1:e_end+51]
        print(f'{e_str}')
        
    elif element_x == 'title':
        print(story['title'])

        
#=======================================================================================
def print_entities(story_x, element_x = None, surface_form_x = None, version_x = None):
    
        element = ''
        if element_x is None or element_x == 'body':
            element = 'body'
        else:
            element = 'title'
        
        # if no surface_form 
        if surface_form_x is None:
            for entity in story_x['entities']:
                pprint(entity)
        
        else:
            
            for entity in story_x['entities']:
                x = 0
                for surface_form in entity[element_x]['surface_forms']:
                    if surface_form_x.lower() in surface_form['text'].lower():
                        x = 1

                if x != 0:
                    pprint(entity)

## 3. Cleaning functions

**Competitors list & Key words**

* Air France Industrie KLM Engineering & Maintenance:  AFI, AFI KLM E&M; KLM E&M, Prognos, Barfield, EPCOR, Spairliners   
* Delta TechOps: Delta, DTO, Delta Services, Delta Material Services (DMS), Digital Alliance  
* Singapore Airlines Engineering Company:  SIAEC, SIAECo, SIA Engineering Company, SAESL, BAPAS; HMS, Eagle Services   
* Turkish Technic:  THY, Turkish Airlines Technic, Habom, Habom MRO Center   
* AJ Walter: AJW, AJ Walter Technique    
* SR Technics:   SRT, SRT Malta  
* ST Aerospace:   STA, ST Engineering, STENG, VT Aerospace  
* StandardAero:  Carlyle, Carlyle Group, Standard Aero  
* AAR:   AAR Corp, StAAR* (Strategic Tools by AAR) , IMOPS* (Inventory Management and Order Processing System), APRISe* (AAR Performance Reporting Information System)    
* Airbus:  Services by Airbus, Skywise, FHS Services, Flight Hour Services, SATAIR  
* Boeing:  Boeing Global Services, BGS, AnalytX, CSP. 
* MTU:  MTU Hannover, MTU Maintenance, MTU Zhuhai, MTU Canada, MTU Brandenburg, MTU Dallas, EME Aero, EME.Aero, MTU Maintenance Serbia  
* GE Aviation: GE Engine Services, GEES, CFM, CFMI, LEAP, GE, GE Celma, GE Wales, PHB, TRUEngines    
* Rolls-Royce: RR, Rolls, Trent, TAY, SPEY, RB211,  Total Care, PBH Total Care    
* Raytheon Technologies:  Pratt&Whitney, Collins Aerospace, PW, P&W, Pratt, Hamilton Sundstrand, UTC, UTAS, United Technologies, Rockwell-Collins, Goodrich, Raytheon     
* SAFRAN: CFMI, CFM, Messier, Dowty, Zodiac, LEAP, SNECMA, Messier-Bugatti-Dowty    

### 3.1 Creating functions to clean the dataframe

In [4]:

# cleaning function for categories

def cleaning_cat(competitors_df):
    
    cat_id_list = []
    cat_label_list = []
    cat_taxonomy_list = []    
    for value in competitors_df['categories']:
        
        try:
            cat_id_list.append(value[0]['id'])
            cat_label_list.append(value[0]['label'])
            cat_taxonomy_list.append(value[0]['taxonomy'])

        except IndexError:
            cat_id_list.append(None)
            cat_label_list.append(None)
            cat_taxonomy_list.append(None)

        
    # convert the results list to Series
    cat_id_column = pd.Series(cat_id_list, dtype=pd.StringDtype())
    cat_label_column = pd.Series(cat_label_list, dtype=pd.StringDtype())
    cat_taxonomy_column = pd.Series(cat_taxonomy_list, dtype=pd.StringDtype())

    # append it to the dataframe & rename the new column 
    clean_df = pd.concat([competitors_df, cat_id_column, cat_label_column, cat_taxonomy_column], axis=1)
    clean_df.rename(columns={0: 'category_id'}, inplace=True)
    clean_df.rename(columns={1: 'category_label'}, inplace=True)
    clean_df.rename(columns={2: 'category_taxonomy'}, inplace=True)
    
    return clean_df

In [5]:
# cleaning country column

def cleaning_location(competitors_df):
    
    """cleaning the source_country column"""
    
    country_list = []
    
    for value in competitors_df['source.locations']:
        try:
            country_list.append(value[0]['country'])
        except IndexError:
            null_value = None
            country_list.append(null_value)

    # convert the results list to Series
    country_column = pd.Series(country_list, dtype=pd.StringDtype())

    # append it to the dataframe, reframe the column & drop the original column
    clean_df = pd.concat([competitors_df, country_column], axis=1)
    clean_df.rename(columns={0: 'source_country'}, inplace=True)    
    
    return clean_df


In [6]:
# cleaning function to replace the body of the article with english version

def replace_body(competitors_df):
    
    """replacing the original news with english translation"""
    body_list = pd.Series([], dtype=pd.StringDtype())
    title_list = pd.Series([], dtype=pd.StringDtype())
    
    for row in range(len(competitors_df)):
        
        try:
            if competitors_df.loc[row, 'language'] != 'en':
                body_list[row] = competitors_df.loc[row, 'translations.en.body']
                title_list[row] = competitors_df.loc[row, 'translations.en.title']
            else:                
                body_list[row] = competitors_df.loc[row, 'body']
                title_list[row] = competitors_df.loc[row, 'title']

        except None: 
            print('y')

    competitors_df.insert(1, "body_en", body_list)
    competitors_df.insert(1, "title_en", title_list)
    
    return competitors_df
    

In [7]:
# choosing columns we need:

def def_cols(competitors_df):
        clean_df = competitors_df[['id',
                                'title_en',
                                'body_en', 
                                'published_at', 
                                'language',
                                'keywords', 
                                'links.permalink', 
                                'source.id', 
                                'source.name',
                                'source_country',
                                'author.id', 
                                'author.name',
                                'sentiment.body.polarity', 
                                'sentiment.body.score',
                                'sentiment.title.polarity', 
                                'sentiment.title.score',
                                'category_id', 
                                'category_label', 
                                'category_taxonomy'
                                ]]
        return clean_df

In [8]:
# function for all functions at once:

def clean_at_once(df):
    competitors_df = cleaning_cat(df)
    competitors_df = cleaning_location(competitors_df)
    competitors_df = replace_body(competitors_df)
    competitors_df = def_cols(competitors_df)
    return competitors_df
        

In [9]:
# function to make id unique with hash

import hashlib

def make_id(competitors_df):
    #new series
    title_id = pd.Series([], dtype=pd.StringDtype())
    body_id = pd.Series([], dtype=pd.StringDtype())
    
    for row in range(len(competitors_df)):

        #convert to bytes
        print(competitors_df['title_en'][row])
        str_title = bytes(competitors_df['title_en'][row], "utf-8")
        str_body = bytes(competitors_df['body_en'][row], "utf-8")
        id_unique = competitors_df['published_at'][row] + '_' + competitors_df['title_en'][row] + '_' + competitors_df['source.name'][row]
        str_id_unique = bytes(id_unique, "utf-8")
        
        # with hash make id
        title_unique_id = hashlib.md5(str_title).hexdigest()
        print('title id ', title_unique_id)
        body_unique_id = hashlib.md5(str_body).hexdigest()
        print('body id ',  body_unique_id)
        unique_id = hashlib.md5(str_id_unique).hexdigest()
        print('unique id ',  unique_id)

        # append values to series
        title_id[row] = title_unique_id
        body_id[row] = body_unique_id
        competitors_df['id'] = unique_id
        

    # insert series in dataframe
    competitors_df.insert(3, 'title_id_unique', title_id)
    competitors_df.insert(2, 'body_id_unique', body_id)
    
    return competitors_df

## 4. Automated search + applied cleaning + uploading to database

In [12]:
# Loop through 90 days and get the correct start & end date with time for each day:

#define date range
days = [*range(0,90,1)]

#define filter
competitors = {
    # 'AFI KLM'     : 'entities:{{element:title AND surface_forms: "Air France Industries KLM Engineering & Maintenance" NOT "football" NOT "tennis" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} entities:{{element:title AND surface_forms: "AFI KLM E&M"}} OR entities:{{element:title AND surface_forms: "KLM E&M"}} OR entities:{{element:title AND surface_forms: "Barfield" AND "MRO"}} OR entities:{{element:title AND surface_forms: "EPCOR" AND "MRO"}} OR entities:{{element:title AND surface_forms: "Spairliners"}} OR entities:{{element:title AND surface_forms: "Air France Industries KLM"}} OR entities:{{element:title AND surface_forms: "Air France KLM"}} OR entities:{{element:title AND surface_forms: "Airfrance KLM"}} OR entities:{{element:title AND surface_forms: "Airfrance Industries KLM"}} OR entities:{{element:title AND surface_forms: "EPCOR" AND "AFI KLM"}}'
     'Delta TechOp'         : 'entities:{{element:title AND surface_forms: "Delta TechOps" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "Delta Services"}} OR entities:{{element:title AND surface_forms: "Delta Material Services"}} OR entities:{{element:title AND surface_forms: "Delta Digital Alliance"}} OR entities:{{element:title AND surface_forms: "Delta" AND "MRO"}}'
    , 'SIAEC'         : 'entities:{{element:title AND surface_forms: "Singapore Airlines Engineering Company" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "SIAEC"}} OR entities:{{element:title AND surface_forms: "SIAECo"}} OR entities:{{element:title AND surface_forms: "SIA Engineering Company"}} OR entities:{{element:title AND surface_forms: "SAESL"}} OR entities:{{element:title AND surface_forms: "BAPAS"}} OR entities:{{element:title AND surface_forms: "Eagle Services"}}'
    , 'Turkish Technic'       : 'entities:{{element:title AND surface_forms: "Turkish Technic" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "Turkish Airlines Technic"}}  OR entities:{{element:title AND surface_forms: "Habom MRO Center"}} OR entities:{{element:title AND surface_forms: "Turkish Technic" AND "MRO"}}'
    , 'AJ Walter'             : 'entities:{{element:title AND surface_forms: "AJ Walter" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "AJW"}} OR entities:{{element:title AND surface_forms: "AJ Walter Technique"}}'
    , 'SR Technics'           : 'entities:{{element:title AND surface_forms: "SR Technics" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "STR Malta"}}'
    , 'ST Aerospace'          : 'entities:{{element:title AND surface_forms: "ST Aerospace" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "ST Engineering"}} OR entities:{{element:title AND surface_forms: "VT Aerospace"}}'
    , 'StandardAero'  : 'entities:{{element:title AND surface_forms: "StandardAero" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "Carlyle Group"}} OR entities:{{element:title AND surface_forms: "Standard Aero"}}'
    , 'AAR'           : 'entities:{{element:title AND surface_forms: "AAR Corp" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "Strategic Tools by AAR"}} OR entities:{{element:title AND surface_forms: "Inventory Management and Order Processing System"}} OR entities:{{element:title AND surface_forms: "AAR Performance Reporting Information System"}}'
    , 'Airbus'        : 'entities:{{element:title AND surface_forms: "Airbus Services" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "Services by Airbus"}} OR entities:{{element:title AND surface_forms: "Skywise"}} OR entities:{{element:title AND surface_forms: "FHS Services"}} OR entities:{{element:title AND surface_forms: "Flight Hour Services"}} OR entities:{{element:title AND surface_forms: "SATAIR"}}'
    , 'MTU'         : 'entities:{{element:title AND surface_forms: "MTU Hannover" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "MTU Maintenance"}} OR entities:{{element:title AND surface_forms: "MTU Zhuhai"}} OR entities:{{element:title AND surface_forms: "MTU Canada"}} OR entities:{{element:title AND surface_forms: "MTU Brandenburg"}} OR entities:{{element:title AND surface_forms: "MTU Dallas"}} OR entities:{{element:title AND surface_forms: "EME Aero"}} OR entities:{{element:title AND surface_forms: "MTU Maintenance Serbia"}} OR entities:{{element:title AND surface_forms: "EME.Aero"}}  OR entities:{{element:title AND surface_forms: "MTU Maintenance Serbia"}} OR entities:{{element:title AND surface_forms: "MTU" AND "MRO"}}'
    , 'GE Aviation'   : 'entities:{{element:title OR surface_forms: "GE Aviation" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "GE Engine Services"}} OR entities:{{element:title AND surface_forms: "GE Celma"}} OR entities:{{element:title AND surface_forms: "GE Wales"}} OR entities:{{element:title AND surface_forms: "TRUEngines"}}'
    , 'Rolls-Royce'   : 'entities:{{element:title AND surface_forms: "Rolls-Royce" NOT "car "NOT "cars" NOT "luxury "NOT "diamonds" NOT "collection" NOT "auto" NOT "autos"  NOT "Ferrari" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "RB211"}} OR entities:{{element:title AND surface_forms: "PBH Total Care"}}'
    , 'Raytheon Technologies'      : 'entities:{{element:title AND surface_forms: "Raytheon Technologies" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "Pratt&Whitney"}} OR entities:{{element:title AND surface_forms: "Collins Aerospace"}} OR entities:{{element:title AND surface_forms: "Hamilton Sundstrand"}} OR entities:{{element:title AND surface_forms: "United Technologies"}} OR entities:{{element:title AND surface_forms: "Rockwell-Collins"}} OR entities:{{element:title AND surface_forms: " Goodrich"}}'
    , 'SAFRAN'        : 'entities:{{element:title AND surface_forms: "Messier-Bugatti-Dowty" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "SNECMA"}} OR entities:{{element:title AND surface_forms: "CFMI" AND "OEM"}}'
    , 'Nayak'         : 'entities:{{element:title AND surface_forms: "Nayak" AND "MRO" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}}' 
    , 'Lufthansa Technik': 'entities:{{element:title AND surface_forms: "Lufthansa Technik" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "Lufthansa Technik AG"}} OR entities:{{element:title AND surface_forms: "Lufthansa Technik" AND "MRO"}}'
    , 'Aerostar':  'entities:{{element:title AND surface_forms: "Aerostar" AND overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR entities:{{element:title AND surface_forms: "Aerostar S.A"}} OR entities:{{element:title AND surface_forms: "Aerostar" AND "MRO"}} OR entities:{{element:title AND surface_forms: "Group Industrial Aeronautic Bacau" AND "MRO"}}'

}

# loop through dates
for day in days:
    published_until = datetime.datetime.today() - datetime.timedelta(days=day)
    published_at_starts = published_until - datetime.timedelta(hours=24)

    published_until = published_until.strftime("%Y-%m-%dT%H:%M:%SZ")
    published_at_starts= published_at_starts.strftime("%Y-%m-%dT%H:%M:%SZ")

    # loop through dict, get jsons, cleanup and store
    for key in competitors:

        #print(published_at_starts, published_until)
        #print(competitors[key])

        params = {
        'published_at.start': published_at_starts
        , 'published_at.end': published_until
        , 'aql': competitors[key]
        , 'per_page' : 100
        ,'translations': {'en'}
        } 

        #get json
        stories = get_top_ranked_stories(params, 500)

        #cleanup json/df
        df = pd.json_normalize(stories)
        if df.empty is False:
                df = clean_at_once(df)
                df['competitor'] = key
                make_id(df)
        else:
            print('empty dataframe')
        
        #store df in DB
        table_name = 'news_filtered'
        schema = 'capstone'
        for row in range(len(df)):
            try:
                df.iloc[row:row+1, :].to_sql(name=table_name, # Name of SQL table
                                con=engine, # Engine or connection
                                if_exists='append', # change to 'append' 
                                schema=schema, # Use schmea that was defined earlier
                                index=False, # Write DataFrame index as a column
                                chunksize=1, # Specify the number of rows in each batch to be written at a time
                                method='multi') # Pass multiple values in a single INSERT clause
                print(f"row {row} of news df was inserted successfully.")
            # Error handling
            except:
                print(f"row {row} of news df was ignored.")
                
        time.sleep(2)
        print('sleeping 2 secs zzzz')
print('Done!')


{'aql': 'entities:{{element:title AND surface_forms: "Delta TechOps" AND '
        'overall_prominence:[0.6 TO *] sort_by(overall_prominence)}} OR '
        'entities:{{element:title AND surface_forms: "Delta Services"}} OR '
        'entities:{{element:title AND surface_forms: "Delta Material '
        'Services"}} OR entities:{{element:title AND surface_forms: "Delta '
        'Digital Alliance"}} OR entities:{{element:title AND surface_forms: '
        '"Delta" AND "MRO"}}',
 'per_page': 100,
 'published_at.end': '2022-06-13T14:02:41Z',
 'published_at.start': '2022-06-12T14:02:41Z',
 'translations': {'en'}}
Visita di manutenzione pesante dell'aeromobile Previsioni di mercato Evergreen Aviation Technologies Corporation (EGAT), Delta TechOps, Honeywell International, Air France-KLM| Valutazione strategica, entrate forti
http://www.brianzadonna.it/notizie/492049/visita-di-manutenzione-pesante-dellaeromobile-previsioni-di-mercato-evergreen-aviation-technologies-corporation-egat-delta-te

In [19]:
#  Competitor: Air France Industries KLM Engineering & Maintenance
#  Key words: AFI, AFI KLM E&M; KLM E&M, Barfield, EPCOR, Spairliners (Key word Prognos left out because it returns sport results)
#  Articles that mention the competitor in the text
#  Last 90 days (maximum scope authorized)

params = {
  'language': ['en'],
  'published_at.start':'NOW-90DAYS',
  'published_at.end':'NOW',
  'text' : 'Safran group" OR ("Safran" AND "OEM" OR "Safran" AND "MRO" OR "CFMI" OR "CFM" or "LEAP" OR "SNECMA" OR "Messier Bugatti Dowty" OR "Safran propulsion" ',
  'cursor': '*',
  'per_page' : 100,
  'sort_by' : 'relevance'
}

stories = get_top_ranked_stories(params, 1000)
stories = pd.DataFrame(stories)

{'cursor': '*',
 'language': ['en'],
 'per_page': 100,
 'published_at.end': 'NOW',
 'published_at.start': 'NOW-90DAYS',
 'sort_by': 'relevance',
 'text': 'Safran group" OR ("Safran" AND "OEM" OR "Safran" AND "MRO" OR "CFMI" '
         'OR "CFM" or "LEAP" OR "SNECMA" OR "Messier Bugatti Dowty" OR "Safran '
         'propulsion" '}
France's Safran tackles impact of Ukraine war but confirms targets By Reuters
https://www.investing.com/news/stock-market-news/frances-safran-tackles-impact-of-ukraine-war-but-confirms-targets-2813655
Fetched 100 stories. Total story count so far: 100
Air France-KLM beats earnings forecasts as summer bookings take off
https://uk.finance.yahoo.com/news/air-france-klm-beats-earnings-051806149.html
Fetched 100 stories. Total story count so far: 200
In-flight Entertainment And Connectivity Global Market Report 2022
https://finance.yahoo.com/news/flight-entertainment-connectivity-global-market-101100589.html
Fetched 100 stories. Total story count so far: 300
Bugatt

In [14]:
stories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author               500 non-null    object
 1   body                 500 non-null    object
 2   categories           500 non-null    object
 3   industries           500 non-null    object
 4   characters_count     500 non-null    int64 
 5   clusters             500 non-null    object
 6   entities             500 non-null    object
 7   hashtags             500 non-null    object
 8   id                   500 non-null    int64 
 9   keywords             500 non-null    object
 10  language             500 non-null    object
 11  links                500 non-null    object
 12  media                500 non-null    object
 13  paragraphs_count     500 non-null    int64 
 14  published_at         500 non-null    object
 15  sentences_count      500 non-null    int64 
 16  sentimen